In [4]:
class LabelEncoder:
    """doc"""

    def __init__(self):
        self.label_mapping = {}
        self.inverse_mapping = {}

    def fit(self, labels):
        """doc"""
        unique_labels = set(labels)
        self.label_mapping = {
            label: index for index, label in enumerate(unique_labels)
        }
        self.inverse_mapping = {
            index: label for label, index in self.label_mapping.items()
        }

    def transform(self, labels):
        """doc"""
        return [self.label_mapping[label] for label in labels]

    def fit_transform(self, labels):
        """doc"""
        self.fit(labels)
        result = self.transform(labels)
        return result

    def inverse_transform(self, encoded_labels):
        """doc"""
        return [self.inverse_mapping[label] for label in encoded_labels]

In [6]:
time1 = time.time()
file = Path(Path().cwd(),'dev.gzip')
main_df = pl.read_parquet(file)
print(len(main_df))
main_df.head()

1000


Log,Target
str,str
""" 1119803499 20…","""normal"""
""" 1119803105 20…","""normal"""
""" 1121496169 20…","""normal"""
""" 1120968564 20…","""normal"""
""" 1120953205 20…","""normal"""


In [7]:
def encoder(data):
    if data == 'normal':
        data = 0
    else:
        data = 1
    return data

def clean_text_pro(text):
    """doc
    """
    text = text.lower()
    text = re.sub('<[^>]*>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    stop_words = set(stopwords.words('english'))
    text = [word for word in text.split() 
            if word not in stop_words and word not in string.punctuation]
    text = ' '.join(word for word in text)
    return text

In [8]:
df = main_df.with_columns(pl.col('Target').apply(encoder, return_dtype=pl.Int32))
df = df.with_columns(pl.col('Log').apply(clean_text_pro))


1000


Log,Target
str,i32
"""rmnccju rmnccj…",0
"""rmnecju rmnecj…",0
"""rmncju rmncju …",0
"""rmncju rmncju …",0
"""rmncju rmncju …",0


In [9]:
time2 = time.time()
print(time2 - time1)

0.27224159240722656


In [13]:
nr  = main_df.filter(pl.col('Target') != 'normal')
print(len(nr))
nr.head()

28


Log,Target
str,str
"""1118542776 200…","""abnormal"""
"""1118540472 200…","""abnormal"""
"""1118543497 200…","""abnormal"""
"""1118541190 200…","""abnormal"""
"""1118542260 200…","""abnormal"""


In [17]:
ab = df.filter(pl.col('Target') == 1)
print(len(ab))
ab

28


Log,Target
str,i32
"""rmncju rmncju …",1
"""rmncju rmncju …",1
"""rmncju rmncju …",1
"""rmncju rmncju …",1
"""rmncju rmncju …",1
"""rmncju rmncju …",1
"""rmncju rmncju …",1
"""rmncju rmncju …",1
"""rmncju rmncju …",1
